In [33]:
import os
import sys
from warcio.archiveiterator import ArchiveIterator

In [37]:
raw_data_dir = '../data/webpages/'
processed_data_dir = '../data/webpages/processed'
os.makedirs(processed_data_dir, exist_ok=True)

In [ ]:
upper_limit = 700000

In [ ]:
i = 0

for filename in os.listdir(raw_data_dir):
    if '.warc' not in filename:
        continue
    full_fname = os.path.join(raw_data_dir, filename)
    with open(full_fname, 'rb') as stream:
        j = 0
        for record in ArchiveIterator(stream):
            if record.rec_type == 'warcinfo':
                print('-------------------')
                print(record.raw_stream.read())
                print('\n-------------------')

            elif record.rec_type == 'response':
                header = record.http_headers.get_header('Content-Type')
                if header is not None and 'text/html' in header:
                    print(record.rec_headers.get_header('WARC-Target-URI'))
                    with open(os.path.join(processed_data_dir, f'{filename}_{j}.txt'), 'wb') as file:
                        file.write(record.content_stream().read())
                    j += 1
                    if i+j > upper_limit:
                        sys.exit()

    i += j

In [ ]:
from comcrawl import IndexClient
client = IndexClient(["2013-48"])
import cdx_toolkit

cdx = cdx_toolkit.CDXFetcher(source='cc')
params = {
    'from_ts': '20131204131715',
    'to': '20131204131716'
}
# index_list = cdx.customize_index_list()
# url = 'commoncrawl.org/*'
url = 'wikipedia.org/*'
# url = 'commoncrawl.org/crawl-data/CC-MAIN-2013-48/*'
print(url, 'size estimate', cdx.get_size_estimate(url, params))

i = 0
for obj in cdx.iter(url, **params, limit=10):
    r = obj.fetch_warc_record()
    stream = obj.content_stream.read()
    # print(stream)
    print(i)
    i += 1